In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [6]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras import layers
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout,AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, GaussianNoise, GlobalMaxPooling2D
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [7]:
#Downloading dataset
!kaggle datasets download -d vuppalaadithyasairam/bone-fracture-detection-using-xrays

100% 172M/172M [00:01<00:00, 142MB/s]
100% 172M/172M [00:01<00:00, 152MB/s]


In [8]:
import zipfile
zip_ref = zipfile.ZipFile('/content/bone-fracture-detection-using-xrays.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [9]:
#Loading the images
x_train= keras.utils.image_dataset_from_directory(
    directory = '/content/archive (6)/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(256,256)
)

x_test= keras.utils.image_dataset_from_directory(
    directory = '/content/archive (6)/val',
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(256,256)
)

Found 8863 files belonging to 2 classes.
Found 600 files belonging to 2 classes.


In [10]:
class_names = x_train.class_names
class_names

['fractured', 'not fractured']

In [11]:
# Normalizing
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

x_train = x_train.map(process)
x_test = x_test.map(process)

In [12]:
input = (256, 256, 3)

In [23]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [22]:
# Creating the base model
base_model = Xception(weights='imagenet', include_top=False)
for layer in base_model.layers:
    layer.trainable = False

83683744/83683744 [==============================] - 0s 0us/step


In [15]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [16]:
model = Model(inputs=base_model.input, outputs=predictions)

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 127, 127, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 127, 127, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 127, 127, 32)         0         ['batch_normalization[0][0

In [18]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [24]:
history = model.fit(x_train,epochs=10, batch_size=100, validation_data=x_test)

Epoch 1/10
89/89 [==============================] - 29s 311ms/step - loss: 0.1703 - accuracy: 0.9405 - val_loss: 0.5401 - val_accuracy: 0.7467
Epoch 2/10
89/89 [==============================] - 32s 354ms/step - loss: 0.1247 - accuracy: 0.9621 - val_loss: 0.5658 - val_accuracy: 0.7467
Epoch 3/10
89/89 [==============================] - 30s 327ms/step - loss: 0.1059 - accuracy: 0.9702 - val_loss: 0.5868 - val_accuracy: 0.7517
Epoch 4/10
89/89 [==============================] - 30s 324ms/step - loss: 0.0917 - accuracy: 0.9746 - val_loss: 0.5910 - val_accuracy: 0.7583
Epoch 5/10
89/89 [==============================] - 31s 326ms/step - loss: 0.0761 - accuracy: 0.9798 - val_loss: 0.6199 - val_accuracy: 0.7600
Epoch 6/10
89/89 [==============================] - 30s 325ms/step - loss: 0.0764 - accuracy: 0.9777 - val_loss: 0.5651 - val_accuracy: 0.7783
Epoch 7/10
89/89 [==============================] - 32s 354ms/step - loss: 0.0575 - accuracy: 0.9857 - val_loss: 0.5757 - val_accuracy: 0.7817